In [31]:
import psycopg2
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
from langdetect import DetectorFactory
from langdetect import detect
from datetime import datetime

DetectorFactory.seed = 0

In [32]:
## Establishing Connection
try:
    conn = psycopg2.connect('''
    dbname='brew' user='rthakur' 
    host='capstone-brewing.cyol5m7lekm6.us-east-1.rds.amazonaws.com' 
    password='brew'
    ''')
except:
    print "Unable to connect to the database"

In [33]:
##Compiling Documents

cur = conn.cursor()
cur.execute("""
select b.id,
       b.name,
       r.text
from wrk.beer b left join wrk.review r on b.id = r.beer_id
where r.beer_id=2886
""")
rows = cur.fetchall()

reviews = []
for row in rows:
    review = row[2].decode('utf-8')
    try:
        if detect(review)=='en':
            reviews.append(review.lower())
    except:
        continue

In [34]:
## Cleaning and Preprocessing

stop = set(stopwords.words('english'))
additional_exclude_words = ['updated','really','good','bad', 'de']
stop |= set(additional_exclude_words)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

reviews_clean = [clean(r).split() for r in reviews]  

True


In [35]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(reviews_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
rev_term_matrix = [dictionary.doc2bow(r) for r in reviews_clean]

In [36]:
print str(datetime.now())
# Creating the object for LDA model
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA on the document term matrix.
ldamodel = Lda(rev_term_matrix, num_topics=5, id2word = dictionary, passes=50)
print str(datetime.now())

2017-07-22 23:03:26.544000
2017-07-22 23:12:07.535000


In [37]:
print str(datetime.now())
print ldamodel.print_topics(num_topics=5, num_words=5)
print str(datetime.now())

2017-07-22 23:12:37.834000
[(0, u'0.031*"dark" + 0.030*"head" + 0.026*"aroma" + 0.026*"brown" + 0.022*"fruit"'), (1, u'0.010*"beer" + 0.006*"chimay" + 0.006*"blue" + 0.005*"trappist" + 0.005*"rating"'), (2, u'0.007*"old" + 0.005*"yeast" + 0.005*"like" + 0.005*"nose" + 0.004*"chocolate"'), (3, u'0.040*"beer" + 0.016*"taste" + 0.015*"one" + 0.012*"flavor" + 0.012*"like"'), (4, u'0.033*"beer" + 0.016*"rated" + 0.014*"trappist" + 0.013*"via" + 0.013*"buddy"')]
2017-07-22 23:12:37.839000
